In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
# For writing data
data_subdirectory = "Data"
if not os.path.exists(data_subdirectory):
    os.makedirs(data_subdirectory)

# Setting the precision for floating points :
np.set_printoptions(precision=4)

In [3]:
write = 0
eve_presence = 0 #'Random'

num_iter = 1    # To average over
in_range = 20    # Range of steps of initial length
ch_noise = 0.02     # 0.00 - 0.30, 0.05 V/s QBER (eve detection)
in_len = [100*(_+1) for _ in range(in_range)]

out_len = np.empty((num_iter, in_range))    # Length of key output for various initial key sizes
time_taken = np.empty((num_iter, in_range)).astype('object')    # avg time taken by a key of a certain length
QBERs = np.empty((num_iter, in_range))
Eve_detected = np.empty((num_iter, in_range))

keys = np.empty((num_iter, in_range))
SKRG = np.empty(in_range)    # Secure key generation rate

In [4]:
total_time = time.time()

for J in range(num_iter):    
    print(J)
    for I in range(in_range):
        print(f"{I=}")
        KEY_LENGTH = in_len[I]

        time_taken[J, I] = time.time()
        %run ./Qiskit_rebuilt_2.ipynb
        time_taken[J, I] = time.time() - time_taken[J, I]
        
        out_len[J, I] += len(key)
        keys[J, I] = str(key)
        QBERs[J, I] = QBER
        Eve_detected[J, I] = ((QBER >= 0.25) and eve) + ((QBER < 0.25) and not eve)    # Wheter or not the DETECTION of Eve is CORRECT
        
    print(f"###################################### Iteration {J} complete ######################################")

total_time = time.time() - total_time
print(total_time)

0
I=0


AttributeError: 'QuantumCircuit' object has no attribute 'qasm'

AttributeError: 'QuantumCircuit' object has no attribute 'qasm'

In [ ]:
print(total_time)
elapsed_time = str(total_time//3600) + "h" + str( (total_time%3600)//60 ) + "m" + str(total_time//60) + "s"
elapsed_time

In [ ]:
# Open the files to write data to
if write :
    filename = f"Data.txt"
    key_strings = f"Keys.txt"
    data_path = os.path.join(data_subdirectory, filename)
    key_path = os.path.join(data_subdirectory, key_strings)
    file = open(data_path, "a")
    file2 = open(key_path, "a")
    
    title = f"\nz = {num_iter}, range = {in_len[0]}-{in_len[-1]}, ch_noise = {ch_noise}, Eve Presence = {eve_presence}, Total Time : {total_time}({elapsed_time})"
    file.write(title)
    file2.write(title)
    
    print("Files updated")

In [ ]:
# SKGR = [sum(1 for i in range(num_iter) if QBERs[i, j] < 0.25 and Eve_detected[i, j])/sum(time_taken[:, j]) for j in range(in_range)]
SKGR = [sum(1 for i in range(num_iter) if QBERs[i, j] < 0.25 and Eve_detected[i, j]) / sum(time_taken[:, j]) for j in range(in_range)]    # Over all iterations

In [ ]:
SKGR

In [ ]:
avg_time_taken = [sum([time_taken[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]
avg_out_len = [sum([out_len[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]
avg_QBERs = [sum([QBERs[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]

if write:
    file.write(f"\nInput Length = [{', '.join(map(str, in_len))}] \nAverage Time Taken = [{', '.join(map(str, avg_time_taken))}] \nAverage QBER = [{', '.join(map(str, avg_QBERs))}] \nAverage Output length = [{', '.join(map(str, avg_out_len))}]\n \nSKGR = [{', '.join(map(str, SKGR))}]")
    file.close()
    file2.write(f"\nKeys = [{', '.join(map(str, keys))}] \n\n Eve detection = [{', '.join(map(str, Eve_detected))}]")
    file2.close()

In [ ]:
fig, ax = plt.subplots(2, 2)

# For key length
coefficients_len = np.polyfit(in_len, avg_out_len, 1)
polynomial_len = np.poly1d(coefficients_len)
equation_len = f'y = {coefficients_len[0]:.2f}x + {coefficients_len[1]:.2f}'
ax[0, 0].text(0.05, 0.95, equation_len, transform=ax[0].transAxes, fontsize=10, verticalalignment='top')
ax[0, 0].set_xlabel('Length of initial key')
ax[0, 0].set_ylabel('Length of final key after Information Reconciliation')
ax[0, 0].set_title("Input vs Output key length")
ax[0, 0].plot(in_len, avg_out_len)
ax[0, 0].plot(in_len, polynomial_len(in_len), linestyle='--')
ax[0, 0].minorticks_on()
ax[0, 0].grid(True)

# For QBER
coefficients_qber = np.polyfit(in_len, avg_QBERs, 1)
polynomial_qber = np.poly1d(coefficients_qber)
equation_qber = f'y = {coefficients_qber[0]:.2f}x + {coefficients_qber[1]:.2f}'
ax[0, 1].text(0.05, 0.95, equation_qber, transform=ax[1].transAxes, fontsize=10, verticalalignment='top')
ax[0, 1].set_xlabel('Length of initial key')
ax[0, 1].set_ylabel('Average QBER')
ax[0, 1].set_title("Input length vs QBER")
ax[0, 1].plot(in_len, avg_QBERs)
ax[0, 1].plot(in_len, polynomial_qber(in_len), linestyle='--')
ax[0, 1].minorticks_on()
ax[0, 1].grid(True)

# For time taken for 1 cycle
coefficients_tt = np.polyfit(in_len, avg_time_taken, 1)
polynomial_tt = np.poly1d(coefficients_tt)
equation_tt = f'y = {coefficients_tt[0]:.2f}x + {coefficients_tt[1]:.2f}'
ax[1, 0].text(0.05, 0.95, equation_tt, transform=ax[2].transAxes, fontsize=10, verticalalignment='top')
ax[1, 0].set_xlabel('Length of initial key')
ax[1, 0].set_ylabel('Time taken for 1 cycle')
ax[1, 0].set_title("Input key length vs time taken")
ax[1, 0].plot(in_len, avg_time_taken)
ax[1, 0].plot(in_len, polynomial_tt(in_len), linestyle='--')
ax[1, 0].minorticks_on()
ax[1, 0].grid(True)

# For Secure key generation rate
coefficients_SKGR = np.polyfit(in_len, SKGR, 1)
polynomial_SKGR = np.poly1d(coefficients_SKGR)
equation_SKGR = f'y = {coefficients_SKGR[0]:.2f}x + {coefficients_SKGR[1]:.2f}'
ax[1, 1].text(0.05, 0.95, equation_SKGR, transform=ax[2].transAxes, fontsize=10, verticalalignment='top')
ax[1, 1].set_xlabel('Length of initial key')
ax[1, 1].set_ylabel('Time taken')
ax[1, 1].set_title("Input key length vs time taken")
ax[1, 1].plot(in_len, SKGR)
ax[1, 1].plot(in_len, polynomial_SKGR(in_len), linestyle='--')
ax[1, 1].minorticks_on()
ax[1, 1].grid(True)

# Set the super title for the entire figure
plt.suptitle(title, weight = 'bold')

# Display the plots
plt.tight_layout()
plt.show()

# Print the title
print(title)


In [ ]:
print(*[item for item in zip(in_len, avg_out_len) if item[1] > 10000], sep = "\n")

In [ ]:
plt.plot(in_len, avg_out_len, label = "Output length")
plt.plot(in_len, avg_QBERs, label = "QBERs")
plt.plot(in_len, avg_time_taken, label = "time_taken")

plt.legend()